In [2]:
import spotipy
import pandas as pd
import numpy as np
import pprint
import sys
from spotipy.oauth2 import SpotifyClientCredentials

In [4]:
file = pd.read_csv("DataWIP.csv",encoding= 'unicode_escape')
sp = spotipy.Spotify(auth_manager= SpotifyClientCredentials(client_id="f854a0450b6b42a28e2ca07467952dd9", client_secret="a03c781ad399487daf947c4a2533bd79"))

#Loading Artists and Albums from CSV
type(file["Artist"])
artist_array = np.array(file["Artist"])
type(file["Album"])
album_array = np.array(file["Album"])

In [5]:
def get_artist(name):
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        return items[0]
    else:
        return None

In [6]:
def get_popularity(name):
    results = sp.search(name, type = 'artist')
    items = results['artists']['items'][0]['popularity']
    return(items)

In [7]:
def get_followers(name):
    results = sp.search(name, type = 'artist')
    items = results['artists']['items'][0]['followers']['total']
    return(items)

In [8]:
def get_albumdate(name):
    results = sp.search(name, type = 'album')
    items = results['albums']['items'][0]['release_date']
    if len(items) > 0:
        return(items)
    else:
        return None

In [9]:
def get_albumcount(name):
    result = sp.search(name) #search query
    result['tracks']['items'][0]['artists']

    #Extract Artist's uri
    artist_uri = get_artisturi(name)

    #Pull all of the artist's albums
    sp_albums = sp.artist_albums(artist_uri, album_type='album')

    album_names = []
    for i in range(len(sp_albums['items'])):
        album_names.append(sp_albums['items'][i]['name'])
    album_dict = list(dict.fromkeys(album_names))
    return len(album_dict)

In [316]:
def get_artisturi(name):
    result = sp.search(q='artist:' + name, type='artist')
    items = result['artists']['items']
    if len(items) > 0:
        return items[0]['uri']
    else:
        return None

In [277]:
uri_list = []
for i in range(len(artist_array)):
    uri_list.append(get_artisturi(artist_array[i]))

In [278]:
follower_list = []

for i in range(0, len(artist_array)):
    if get_followers(artist_array[i]) is not None:
        follower_list.append(get_followers(artist_array[i]))
    else:
        follower_list.append(None)

In [279]:
popularity_list = []

for i in range(0, len(artist_array)):
    if get_popularity(artist_array[i]) is not None:
        popularity_list.append(get_popularity(artist_array[i]))
    else:
        popularity_list.append(None)

In [285]:
count_list = []

for i in range(0, len(artist_array)):
    if get_albumcount(artist_array[i]) is not None:
        count_list.append(get_albumcount(artist_array[i]))
    else:
        count_list.append(None)

In [291]:
#CSV File
dict = {'AlbumCount': count_list,'URI': uri_list, 'SpotifyFollowers': follower_list, 'SpotifyPopularity': popularity_list}#, 'AlbumDate': albumdate_list}
df = pd.DataFrame(dict)
df.to_csv('Spotify.csv')